# Standard packages

In [1]:
import os
import pandas as pd
import time

import numpy as np
import tensorflow as tf

# disable warning TF logs
tf.get_logger().setLevel('ERROR')

# Specific installs

rna-state-inf is a pre-trained TensorFlow model.

In [12]:
method_name  = "rna-state-inf"

model_fname  = 'rna-state-inf_rnnepoch50.h5'
gtfold_fname = 'gtfold-3.0_x86-64_ubuntu'
rs_fname     = 'RNAstructureLinuxTextInterfaces64bit.tgz'

url1 = f"https://raw.githubusercontent.com/sinc-lab/lncRNA-folding/main/methods/{model_fname}"
url2 = f"https://master.dl.sourceforge.net/project/gtfold/{gtfold_fname}.tar.gz?viasf=1"
url3 = f"http://rna.urmc.rochester.edu/Releases/current/{rs_fname}"

In [13]:
# download trained model used for estimating SHAPE data. It was trained following the instructions in this repository:
# https://github.com/dwillmott/rna-state-inf
!wget -q {url1}

In [14]:
# download method to predict secondary structure using SHAPE data
!wget -q {url2} -O {gtfold_fname}.tar.gz
!tar xfz {gtfold_fname}.tar.gz

gfold_data = os.path.join(gtfold_fname, 'data')
gfold_bin  = os.path.join(gtfold_fname, 'gtmfe')

!{gfold_bin} | head -3

GTfold: A Scalable Multicore Code for RNA Secondary Structure Prediction
(c) 2007-2011  D.A. Bader, C.E. Heitsch, S.C. Harvey
Georgia Institute of Technology


In [15]:
# download utility for converting ct files to dot files
!wget -q {url3}
!tar xfz {rs_fname}

source_path = "/content/RNAstructure/"
data_table  = os.path.join(source_path, 'data_tables')
ct2dot_bin  = os.path.join(source_path, 'exe', 'ct2dot')

!{ct2dot_bin} --version

ct2dot: Version 6.4 (December 8, 2021).
Copyright Mathews Lab, University of Rochester.
Invalid structure number given.


# S. cerevisiae (sce) 18 long non-coding RNA dataset
Data source: https://genie.weizmann.ac.il/pubs/PARS10/data/sce_genes_folded.tab.gz

In [16]:
gh_path = "https://raw.githubusercontent.com/sinc-lab/lncRNA-folding/master/data/"
sce = pd.read_csv(gh_path + "sce_genes_folded.tab", delimiter='\t', 
                  header=None, index_col=0, 
                  names=("Gene ID", "sequence", "PARS-assisted folding"))

In [17]:
# Sequences to process
sce18 = ["snR81", "snR34", "snR43", "snR44",  "snR31",  "snR10",
         "snR63", "snR11", "snR82", "snR17b", "snR17a", "snR37",
         "SCR1",  "SRG1",  "snR19", "snR30",  "LSR1",   "TLC1"]

# Compute structures

In [18]:
#@title Utils
sequencedict = {'A' : 0, 'C' : 1, 'G' : 2, 'U' : 3}
for letter in 'BDEFHIJKLMNOPQRSTVWXYZ':
    sequencedict.update({letter : 4})

def process_sequence(fin):
    with open(fin) as f:
        for a in f:
            _, seq, _ = a.strip().split('\t')

    seq = [sequencedict[s] for s in list(seq)]
    seq = tf.one_hot(seq, 5)
    seq = tf.convert_to_tensor([seq])

    return seq

def prob2shape(lstm_probs):
    """This method was taken from the following link: 
    https://github.com/dwillmott/rna-state-inf/blob/master/method.py
    """
    a = 0.214
    b = 0.6624
    s = 0.3603

    outstr = ""
    #for each position, write a SHAPE value depending on the probability
    for i, lstm_prob in enumerate(lstm_probs):
        if lstm_prob >= 0.5:
            shapevalue = ((a-s)/0.5)*(lstm_prob-1)+a
        else:
            shapevalue = ((s-b)/0.5)*lstm_prob+b

        outstr += '%d %.3f ' % (i+1, shapevalue)
        outstr += '\n'

    return outstr

def make_prediction(seq_fin, model_fin, outfname):
    seq   = process_sequence(seq_fin)
    model = tf.keras.models.load_model(model_fin)

    pred  = model.predict(seq)
    #np.savetxt(sys.stdout.buffer, pred[0,:], delimiter='\t')
    strout = prob2shape(pred[0,:,1])

    fout = open(outfname, 'w')
    fout.write(strout)
    fout.close()

def fas2tsv(fin):
  seq = ""
  with open(fin) as f:
    for a in f:
      b = a.strip()
      if b[0] == ">":
        desc = b[1:]
      else:
        seq += b
  
  # write files
  foutname = fin + '.tsv'
  out = open(foutname, 'w')
  out.write('{}\t{}\t{}\n'.format(desc, seq, desc))
  out.close()

  foutseq = fin + '.seq'
  out = open(foutseq, 'w')
  out.write('{}\n'.format(seq))
  out.close()

  return foutname, foutseq

In [19]:
def run_folding(fasta_name):
  out_file_name = fasta_name + '.dot'
  tmp_file1 = out_file_name + '.tmp1'
  tmp_file2 = out_file_name + '.tmp2'

  tsv_file, seq_file = fas2tsv(fasta_name)

  # estimate SHAPE-like data
  make_prediction(tsv_file, model_fname, tmp_file1)
  
  # predict structure from estimated SHAPE data
  os.system(f"export GTFOLDDATADIR={gfold_data}; {gfold_bin} {seq_file} --useSHAPE {tmp_file1} --output {out_file_name} > /dev/null 2>&1")
  
  # convert ct to dot-bracket format
  os.system(f"export DATAPATH={data_table}; {ct2dot_bin} {out_file_name + '.ct'} -1 {tmp_file2} >/dev/null 2>&1")

  # fix description line
  os.system(f"head -1 {fasta_name}   > {out_file_name} # original description line")
  os.system(f"tail -n+2 {tmp_file2} >> {out_file_name} # sequence + predicted structure")

  return out_file_name

In [20]:
out_fasta_name = method_name + "_yeast18"
if os.path.exists(out_fasta_name + ".fasta"): os.remove(out_fasta_name + ".fasta")

print("   \t lnc \t len \t time")
for i, lnc in enumerate(sce18): 

  start_time = time.time()
  seq = sce.loc[lnc]["sequence"]
  print(f"{i+1}/{len(sce18)}\t{lnc} \t {len(seq)}", end='\t')

  # Write a one-sequence fasta
  with open("tmp.fasta", "w") as ofile: 
    ofile.write(f">{lnc}\n{seq}\n")
  
  dot_file_name = run_folding("tmp.fasta")

  # Concatenate outputs
  os.system("cat " + dot_file_name + " >> " + out_fasta_name + ".fasta") 

  print(f"{time.time() - start_time: .1f} s")

   	 lnc 	 len 	 time
1/18	snR81 	 201	 3.5 s
2/18	snR34 	 203	 3.8 s
3/18	snR43 	 209	 3.0 s
4/18	snR44 	 211	 3.3 s
5/18	snR31 	 225	 3.0 s
6/18	snR10 	 245	 3.4 s
7/18	snR63 	 255	 3.1 s
8/18	snR11 	 258	 3.0 s
9/18	snR82 	 268	 3.5 s
10/18	snR17b 	 332	 3.1 s
11/18	snR17a 	 333	 3.4 s
12/18	snR37 	 386	 3.2 s
13/18	SCR1 	 522	 3.3 s
14/18	SRG1 	 551	 3.8 s
15/18	snR19 	 568	 3.4 s
16/18	snR30 	 606	 3.8 s
17/18	LSR1 	 1175	 5.1 s
18/18	TLC1 	 1301	 5.9 s
